## main.py

1. import all necessary modules

In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.preprocessing import image
from imblearn.over_sampling import SMOTE

sys.path.insert(0, os.path.abspath('./utils/'))
import util as utils

Using TensorFlow backend.


In [2]:
def get_model():
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPool2D((2, 2)))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(9, activation='softmax'))
    
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', 
                  metrics=['acc'])
    
    return model

In [3]:
get_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        36928     
__________

In [4]:
dataset_dir = os.path.abspath('./dataset/')

train_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

In [5]:
train_files = [ os.path.splitext(file)[0] for file in os.listdir(train_dir) ]
validation_files = [ os.path.splitext(file)[0] for file in os.listdir(validation_dir) ]
test_files = [ os.path.splitext(file)[0] for file in os.listdir(test_dir) ]

In [6]:
data_labels = { filename: label for filename, label in pd.read_csv('./trainLabels.csv').get_values() }

In [7]:
train_labels = [ data_labels.get(filename, filename) for filename in train_files ]
validation_labels = [ data_labels.get(filename, filename) for filename in validation_files ]
test_labels = [ data_labels.get(filename, filename) for filename in test_files ]

In [8]:
train_set = [ utils.load_image_as_np(os.path.join(train_dir, (filename+'.png'))) for filename in train_files ]
validation_set = [ utils.load_image_as_np(os.path.join(validation_dir, (filename+'.png'))) for filename in validation_files ]
test_set = [ utils.load_image_as_np(os.path.join(test_dir, (filename+'.png'))) for filename in test_files ]

In [9]:
train_set = np.asarray(train_set)
validation_set = np.asarray(validation_set)
test_set = np.asarray(test_set)

In [10]:
train_set = train_set.astype('float32') / 255
validation_set = validation_set.astype('float32') / 255
test_set = test_set.astype('float32') / 255

In [11]:
print('train_set.shape:       ', train_set.shape)
print('validation_set.shape:  ', validation_set.shape)
print('test_set.shape:        ', test_set.shape)

train_set.shape:        (4352, 128, 128, 1)
validation_set.shape:   (3258, 128, 128, 1)
test_set.shape:         (3258, 128, 128, 1)


In [12]:
train_labels = to_categorical(train_labels)
validation_labels = to_categorical(validation_labels)
test_labels = to_categorical(test_labels)

In [13]:
train_labels.shape

(4352, 10)

In [14]:
sm = SMOTE()

In [15]:
train_set_res, train_labels_res = sm.fit_resample(train_set.reshape(train_set.shape[0], (128 * 128 * 1)), train_labels)

In [16]:
train_set_res = train_set_res.reshape((train_set_res.shape[0], 128, 128, 1))

In [17]:
print('train_set_res.shape:     ', train_set_res.shape)
print('train_labels_res.shape:  ', train_labels_res.shape)

train_set_res.shape:      (11088, 128, 128, 1)
train_labels_res.shape:   (11088, 9)


In [18]:
validation_labels = np.delete(validation_labels, 0, 1)

In [19]:
model = get_model()

In [20]:
model.fit(train_set_res, train_labels_res, batch_size=128, epochs=10, validation_data=(validation_set, validation_labels))

Train on 11088 samples, validate on 3258 samples
Epoch 1/10
11088/11088 [==============================] - 118s 11ms/step - loss: 1.2575 - acc: 0.5951 - val_loss: 0.7090 - val_acc: 0.7502
Epoch 2/10
11088/11088 [==============================] - 103s 9ms/step - loss: 1.5762 - acc: 0.7182 - val_loss: 8.2655 - val_acc: 0.4791
Epoch 3/10
11088/11088 [==============================] - 103s 9ms/step - loss: 4.8487 - acc: 0.6544 - val_loss: 3.3154 - val_acc: 0.7655
Epoch 4/10
11088/11088 [==============================] - 103s 9ms/step - loss: 9.7002 - acc: 0.3954 - val_loss: 14.8846 - val_acc: 0.0764
Epoch 5/10
11088/11088 [==============================] - 103s 9ms/step - loss: 14.3126 - acc: 0.1120 - val_loss: 16.0340 - val_acc: 0.0052
Epoch 6/10
11088/11088 [==============================] - 106s 10ms/step - loss: 14.3272 - acc: 0.1111 - val_loss: 16.0340 - val_acc: 0.0052
Epoch 7/10
11088/11088 [==============================] - 103s 9ms/step - loss: 14.3272 - acc: 0.1111 - val_loss: 16